#### Import necessary Library

In [4]:
import urllib.request
import json
import ssl
import os
from dotenv import load_dotenv

In [5]:
from dotenv import dotenv_values
config = dotenv_values("config.env")
config=dict(config)
api_key = config["api_key_completion"]
url = config["url_completion"]

•  Conditional Check: The function starts by checking if both allowed is True and the PYTHONHTTPSVERIFY environment variable is not set or is an empty string. This variable, if present and non-empty, controls HTTPS certificate verification behavior.


•  ssl Module Attribute: It then checks if the ssl module has an attribute named _create_unverified_context. This attribute, if present, provides a way to create an HTTPS context without verifying the server's certificate.

•  Modification: If both conditions are met (allowing self-signed certs and unverified context creation), the function sets the ssl._create_default_https_context attribute to the _create_unverified_context value. This effectively instructs the ssl module to skip server certificate verification for all HTTPS connections.


In [6]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

•this line is needed if you use self-signed certificate in your scoring service.

In [7]:
allowSelfSignedHttps(True)

•	prompt: This is the starting text or phrase that the LLM will use to begin generating new text.

•	temperature: This parameter controls the "creativity" or randomness of the output.

•	max_tokens: This parameter specifies the maximum number of tokens (words or sub-word units) that the LLM should generate for the output.


In [8]:
data =  {
  "prompt": "My name is Julien and I like to",
  "temperature": 0.8,
  "max_tokens": 128
}

•	json.dumps(data): This function takes a Python object (data) and converts it into a JSON string. JSON (JavaScript Object Notation) is a popular format for exchanging data between applications.

•	str.encode(json_string): This function takes the JSON string generated in the previous step and encodes it into bytes. This encoding is necessary because many network protocols, like HTTP, expect data to be sent as bytes.


In [9]:
body = str.encode(json.dumps(data))

• Load the environmet variable from the .env file

• Load the URL from the .env file using this os.getenv()

• Load the primary/secondary key or AMLToken for the endpoint from the .env file using os.getenv()

•Checking api key is correct or not 

In [11]:
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

•Content-Type: Identifies the data format as JSON.

•	Authorization: Authenticates the request using your API key.

In [15]:
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

•	Below code will send the request and return a response object containing the server's response data and headers.

In [16]:
req = urllib.request.Request(url, body, headers)

•response = urllib.request.urlopen(req): This line sends the HTTP request using the req object (which should contain the URL, body, and headers).

•result = response.read(): If the request is successful, this line reads the entire response body into the result variable.

•	except urllib.error.HTTPError as error: This part defines an except block to handle HTTPError exceptions that might occur during the request. The error object is stored as error.

In [22]:
try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

The request failed with status code: 424
Content-Length: 95
Content-Type: text/plain
x-request-id: 5807609d-74e1-486b-8bdf-c667bac2f2e9
ms-azureml-model-error-reason: model_not_ready
Request-Context: appId=cid-v1:b80100f5-1286-423d-967f-3517f665186d
azureml-model-session: dep-2-70b-231109102345
azureml-model-group: offer-llama-2-70b-paygo
Date: Thu, 04 Apr 2024 05:29:26 GMT
Connection: close


upstream connect error or disconnect/reset before headers. reset reason: connection termination
